In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss

In [125]:
# train = pd.read_pickle(FILE.train_agg_v1.value)
train = pd.read_pickle(FILE.train_base_line.value)
print('train shape is: {}'.format(train.shape))
# test = pd.read_pickle(FILE.test_agg_v1.value)
test = pd.read_pickle(FILE.test_base_line.value)
print('test shape is: {}'.format(test.shape))

# train = train1.iloc[:,:200].copy()
# train['click'] = train1['click'].copy()
# test = test1.iloc[:,:200].copy()

train shape is: (1001650, 33)
test shape is: (40024, 32)


# LGBM

In [9]:
from sklearn.model_selection import train_test_split
print_to_file = False 
test_run = False 

def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)

    return est.strftime(time_format)

import sys, time
class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
        #         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('[' + get_time() + '] ' + message)
            self.log.close()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

if print_to_file:
    divert_printout_to_file()  # note: comment this to use pdb

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, train_df, test_df, holdout, num_folds, submission_file_name, fe_img_name, stratified = False, debug= False, colsample=0.67, max_depth=8, num_leaves=31, min_child_samples=20, subsample=0.7, reg_lambda=0.3, lr=0.04, seed=1001, verbose=100, rounds=None, target='click'):
    print(train_df.shape, test_df.shape, holdout.shape)
    print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df[target].mean(), holdout[target].mean())
    # Divide in training/validation and test data
    if df is not None:
        train_df = df[df[target].notnull()]
        test_df = df[df[target].isnull()]
        print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
        del df
        gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    holdout_final_preds = np.zeros(holdout.shape[0])
    feature_importance_df = pd.DataFrame()
    feature_importance_gain_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in [target,'time','instance_id','index']]
    train_scores = []
    holdout_scores = []
    scores = []
    diff_val_holdout = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df[target])):
#         print('valid index : ',list(valid_idx)[:5])
        train_x, train_y = train_df[feats].iloc[train_idx], train_df[target].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df[target].iloc[valid_idx]
#         print('MEAN: train({}) vs valid({}): '.format(len(train_y), len(valid_y)), np.mean(train_y), np.mean(valid_y))

        clf = LGBMClassifier(
            nthread=10,
            n_estimators=30000,
            learning_rate=lr,
#             max_bin =10000,
#             num_leaves=num_leaves,
#             colsample_bytree=colsample, # 0.67
#             subsample=subsample,
#             subsample_freq=0, ## disable subsampling
#             max_depth=max_depth,
#             reg_alpha=0.65,
#             reg_lambda=reg_lambda,
#             min_split_gain=0.0222415,
#             min_child_weight=39.3259775,
#             min_child_samples=min_child_samples,
            silent=-1,
            verbose=-1, )
        if rounds is not None:
            clf.n_estimators = rounds
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose)
            oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats])[:, 1] 
        else:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose, early_stopping_rounds= 200)
            oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats], num_iteration=clf.best_iteration_)[:, 1] 
            
        holdout_final_preds += holdout_preds / folds.n_splits
        score = log_loss(valid_y, oof_preds[valid_idx])
        train_score = clf.best_score_['training']['binary_logloss']
        holdout_score = log_loss(holdout[target], holdout_preds)
        diff = abs(score - holdout_score)
        best_rounds = rounds if rounds is not None else clf.best_iteration_
        print('Fold %2d [%5d] AUC : ho: %.6f / te: %.6f / tr: %.6f (diff: %.6f)' % (n_fold + 1, best_rounds, holdout_score, score,  train_score, diff))
        scores.append(score)
        train_scores.append(train_score)
        holdout_scores.append(holdout_score)
        diff_val_holdout.append(diff)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        fold_importance_gain_df = pd.DataFrame()
        fold_importance_gain_df["feature"] = feats
        fold_importance_gain_df["importance"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance_gain_df["fold"] = n_fold + 1
        feature_importance_gain_df = pd.concat([feature_importance_gain_df, fold_importance_gain_df], axis=0)
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    holdout_roc = log_loss(holdout[target], holdout_final_preds)
    holdout_mean = np.mean(holdout_scores)
    full_te_mean = np.mean(scores)
    full_tr_mean = np.mean(train_scores)
#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('Full HO score %.6f' % holdout_roc)
    print('FULL HO mean {:.6f}, std {:.6f}'.format(holdout_mean, np.std(holdout_scores)))
    print('FULL TE mean {:.6f}, std {:.6f}'.format(full_te_mean, np.std(scores)))
    print('FULL TR mean {:.6f}, std {:.6f}'.format(full_tr_mean, np.std(train_scores)))
    print('FULL DIFF mean {:.6f}, std {:.6f}'.format(np.mean(diff_val_holdout), np.std(diff_val_holdout)))
    # Write submission file and plot feature importance
    if not debug:
        print('saving...')
        test_df['predicted_score'] = sub_preds
        test_df[['instance_id', 'predicted_score']].to_csv(submission_file_name, index= False)
#     if not print_to_file:
#         display_importances(feature_importance_df, fe_img_name)
    feature_importance_df = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    feature_importance_gain_df = feature_importance_gain_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    return feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds

# Display/plot feature importance
def display_importances(feature_importance_df_, fe_img_name):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(fe_img_name+'.png')


def convert_and_save_imp_df(fe_imp_df, dumpfilename):
    fe_imp_df_mean = fe_imp_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    pickle.dump(fe_imp_df_mean, open(dumpfilename,'wb'))
    
    

def select_holdout(train,test,shift=1,seed=6):
    samples_size = int(len(test) )
    assert shift in range(1,8)
    test_time_min = test.time.min()
    test_time_max = test.time.max()
    train_time_min = train.time.min()
    train_time_max = train.time.max()
    window_lower_bound = test_time_min - shift*60*60*24
    window_upper_bound = test_time_max - shift*60*60*24
    available_train = train[(train.time<=window_upper_bound) & (train.time>=window_lower_bound)]
    holdout = available_train.sample(n=samples_size,random_state=seed).copy()
    train_split_index = list(set(train.index) - set(holdout.index))
    train_split = train.iloc[train_split_index].copy()
    return train_split,holdout

In [27]:
def runlgb(train, test, holdout):
    colsamples = [0.07]#[0.1,0.15,0.2]#[0.03,0.04,0.05,0.06,0.07,0.08]
    seeds = [20]#[300,4000,50000,600000,7000000,80000000,523445,31275479] # 20
    depth = [5]
    leaves = [16]
    min_child_sam = [20]#, 800]
    subsamples = [1]#0.8, 0.7, 0.6, 0.5, 0.4] # was 1
    reg_lambdas = [0.5]
    # lrs = lrs.tolist()
    lrs2 = [0.05]
    nfolds = 5
    rounds = [None] #[1000]#, 1300, 1600, 1900, 2200, 2500]
    for seed in seeds:
        for colsample in colsamples:
            for d in depth:
                for l in leaves:
                    for mcs in min_child_sam:
                        for subsample in subsamples:
                            for reg_lambda in reg_lambdas:
                                for lr in lrs2:
                                    for r in rounds:
                                        filename = 'fe_936_col{}_lr{}_n{}'.format(len(train.columns), lr, nfolds)
                                        print('#############################################')
                                        print(colsample, seed, d, l, mcs, subsample, reg_lambda, lr, 'nfolds:', nfolds)
                                        print('#############################################')
                                        numfeats = len(train.columns)
                                        with timer("Run LightGBM with kfold"):
                                            return kfold_lightgbm(None, train, test, holdout, nfolds, filename+'.csv', filename, colsample=colsample, verbose=None, max_depth=d, num_leaves=l, min_child_samples=mcs, subsample=subsample, reg_lambda=reg_lambda, lr=lr, seed=seed, stratified=True, rounds=r,debug=False)                                        

In [126]:
train_df,holdout = select_holdout(train,test,shift=1,seed=41)
print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
print('train mean: {}, holdout mean: {}'.format(train.click.mean(),holdout.click.mean()))
feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test, holdout)

train shape: (961626, 33), holdout shape: (40024, 33)
train mean: 0.19845954175610242, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 33) (40024, 32) (40024, 33)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  471] AUC : ho: 0.427171 / te: 0.416487 / tr: 0.409813 (diff: 0.010684)
Fold  2 [  626] AUC : ho: 0.426880 / te: 0.415591 / tr: 0.408073 (diff: 0.011288)
Fold  3 [  721] AUC : ho: 0.427193 / te: 0.416444 / tr: 0.406716 (diff: 0.010749)
Fold  4 [  438] AUC : ho: 0.426814 / te: 0.417162 / tr: 0.410096 (diff: 0.009653)
Fold  5 [  617] AUC : ho: 0.427204 / te: 0.416090 / tr: 0.408036 (diff: 0.011114)
Full HO score 0.426595
FULL HO mean 0.427052, std 0.000169
FULL TE mean 0.416355, std 0.000516
FULL TR mean 0.408546, std 0.001252
FULL DIFF mean 0.010698, std 0.000569
saving...
Run LightGBM with kfold - done in 87s


# Select features

In [127]:
base_gain = feature_importance_gain_df.copy()
base_split = feature_importance_df.copy()

In [14]:
base_split.to_csv('../../data/features/feature_importance_split_baseline.csv',index=False)
base_gain.to_csv('../../data/features/feature_importance_gain_baseline.csv',index=False)

# get train holdout
shift day may change from 1 to 7

In [115]:
train.dtypes

instance_id                 int64
time                        int64
city                      float64
province                  float64
user_tags                 float64
carrier                     int64
devtype                     int64
make                      float64
model                     float64
nnt                         int64
os                          int64
osv                       float64
os_name                    object
adid                        int64
advert_id                   int64
orderid                     int64
advert_industry_inner    category
campaign_id                 int64
creative_id                 int64
creative_tp_dnf             int64
app_cate_id              category
f_channel                category
app_id                   category
inner_slot_id             float64
creative_type               int64
creative_width           category
creative_height          category
creative_is_jump            int64
creative_is_download        int64
creative_has_d

In [128]:
base_split

,feature,importance
0,app_id,2893.2
1,model,2322.0
2,user_tags,2099.4
3,city,1954.6
4,osv,1355.0
5,time_hour,1220.4
6,make,813.6
7,adid,738.0
8,creative_id,640.2
9,inner_slot_id,550.6


In [73]:
# seed = 6
seed = 41
def select_holdout(train,test,shift=1,seed=6):
    samples_size = int(len(test) )
    assert shift in range(1,8)
    test_time_min = test.time.min()
    test_time_max = test.time.max()
    train_time_min = train.time.min()
    train_time_max = train.time.max()
    window_lower_bound = test_time_min - shift*60*60*24
    window_upper_bound = test_time_max - shift*60*60*24
    available_train = train[(train.time<=window_upper_bound) & (train.time>=window_lower_bound)]
    holdout = available_train.sample(n=samples_size,random_state=seed).copy()
    train_split_index = list(set(train.index) - set(holdout.index))
    train_split = train.iloc[train_split_index].copy()
    return train_split,holdout

In [76]:
holdout_score_list = []
for shift in range(1,2):
    for seed in range(1,50):
        print('shift hold out: {}'.format(shift))
        print('seed is: {}'.format(seed))
        train_df,holdout = select_holdout(train,test,shift=shift,seed=seed)
        feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test, holdout)
        holdout_score_list.append(holdout_roc)
        print('............')

shift hold out: 1
seed is: 1
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 41) (40024, 42) (40024, 41)
MEAN: train(961626) vs holdout(40024):  0.19797509634722854 0.21009894063561863
Fold  1 [  106] AUC : ho: 0.428380 / te: 0.417820 / tr: 0.409564 (diff: 0.010561)
Fold  2 [  109] AUC : ho: 0.427984 / te: 0.417523 / tr: 0.409473 (diff: 0.010461)
Fold  3 [  110] AUC : ho: 0.428035 / te: 0.418154 / tr: 0.409424 (diff: 0.009880)
Fold  4 [  113] AUC : ho: 0.428350 / te: 0.417215 / tr: 0.409275 (diff: 0.011135)
Fold  5 [  122] AUC : ho: 0.428154 / te: 0.418153 / tr: 0.408387 (diff: 0.010002)
Full HO score 0.427635
FULL HO mean 0.428181, std 0.000161
FULL TE mean 0.417773, std 0.000365
FULL TR mean 0.409224, std 0.000429
FULL DIFF mean 0.010408, std 0.000447
Run LightGBM with kfold - done in 58s
............
shift hold out: 1
seed is: 2
#############################################
0.07 20 5 16 20 1 0.

KeyboardInterrupt: 

In [74]:
for shift in range(1,2):
    print('shift hold out: {}'.format(shift))
    train_df,holdout = select_holdout(train,test,shift=shift)
    feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test, holdout)
    print('............')

shift hold out: 1
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 41) (40024, 42) (40024, 41)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  124] AUC : ho: 0.428527 / te: 0.418169 / tr: 0.408282 (diff: 0.010358)
Fold  2 [  101] AUC : ho: 0.427919 / te: 0.417134 / tr: 0.410185 (diff: 0.010785)
Fold  3 [  114] AUC : ho: 0.428193 / te: 0.417614 / tr: 0.409079 (diff: 0.010579)
Fold  4 [  111] AUC : ho: 0.428110 / te: 0.418588 / tr: 0.409075 (diff: 0.009522)
Fold  5 [   98] AUC : ho: 0.428226 / te: 0.417259 / tr: 0.410338 (diff: 0.010966)
Full HO score 0.427649
FULL HO mean 0.428195, std 0.000197
FULL TE mean 0.417753, std 0.000551
FULL TR mean 0.409392, std 0.000769
FULL DIFF mean 0.010442, std 0.000503
Run LightGBM with kfold - done in 60s
............


In [71]:
for shift in range(1,8):
    print('shift hold out: {}'.format(shift))
    train_df,holdout = select_holdout(train,test,shift=shift)
    feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test, holdout)
    print('............')

shift hold out: 1
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(981638, 41) (40024, 42) (20012, 41)
MEAN: train(981638) vs holdout(20012):  0.19823804701936967 0.20932440535678593
Fold  1 [  114] AUC : ho: 0.426128 / te: 0.418824 / tr: 0.409337 (diff: 0.007303)
Fold  2 [  119] AUC : ho: 0.426505 / te: 0.417971 / tr: 0.409058 (diff: 0.008534)
Fold  3 [  106] AUC : ho: 0.425851 / te: 0.417961 / tr: 0.410074 (diff: 0.007890)
Fold  4 [  104] AUC : ho: 0.426432 / te: 0.417768 / tr: 0.410181 (diff: 0.008664)
Fold  5 [  106] AUC : ho: 0.426169 / te: 0.417312 / tr: 0.410173 (diff: 0.008857)
Full HO score 0.425668
FULL HO mean 0.426217, std 0.000234
FULL TE mean 0.417967, std 0.000491
FULL TR mean 0.409764, std 0.000473
FULL DIFF mean 0.008250, std 0.000574
Run LightGBM with kfold - done in 59s
............
shift hold out: 2
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#####

In [42]:
for shift in range(1,8):
    print('shift hold out: {}'.format(shift))
    train_df,holdout = select_holdout(train,test,shift=shift)
    feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train, test, holdout)
    print('............')

shift hold out: 1
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(1001650, 41) (40024, 40) (20012, 41)
MEAN: train(1001650) vs holdout(20012):  0.19845954175610242 0.20927443533879672
Fold  1 [  106] AUC : ho: 0.419230 / te: 0.419414 / tr: 0.409997 (diff: 0.000184)
Fold  2 [  117] AUC : ho: 0.418970 / te: 0.416817 / tr: 0.409754 (diff: 0.002153)
Fold  3 [  125] AUC : ho: 0.418135 / te: 0.417705 / tr: 0.409009 (diff: 0.000430)
Fold  4 [  122] AUC : ho: 0.418828 / te: 0.418672 / tr: 0.409032 (diff: 0.000155)
Fold  5 [  103] AUC : ho: 0.419829 / te: 0.417820 / tr: 0.410590 (diff: 0.002009)
Full HO score 0.418336
FULL HO mean 0.418998, std 0.000551
FULL TE mean 0.418086, std 0.000887
FULL TR mean 0.409676, std 0.000601
FULL DIFF mean 0.000986, std 0.000900
Run LightGBM with kfold - done in 62s
............
shift hold out: 2
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
###

KeyboardInterrupt: 

In [40]:
for shift in range(1,8):
    print('shift hold out: {}'.format(shift))
    train_df,holdout = select_holdout(train,test,shift=shift)
    feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train, test, holdout)
    print('............')

shift hold out: 1
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(1001650, 41) (40024, 40) (40024, 41)
MEAN: train(1001650) vs holdout(40024):  0.19845954175610242 0.21007395562662404
Fold  1 [  106] AUC : ho: 0.420683 / te: 0.419414 / tr: 0.409997 (diff: 0.001269)
Fold  2 [  117] AUC : ho: 0.419598 / te: 0.416817 / tr: 0.409754 (diff: 0.002782)
Fold  3 [  125] AUC : ho: 0.419518 / te: 0.417705 / tr: 0.409009 (diff: 0.001813)
Fold  4 [  122] AUC : ho: 0.419512 / te: 0.418672 / tr: 0.409032 (diff: 0.000840)
Fold  5 [  103] AUC : ho: 0.420665 / te: 0.417820 / tr: 0.410590 (diff: 0.002845)
Full HO score 0.419337
FULL HO mean 0.419995, std 0.000555
FULL TE mean 0.418086, std 0.000887
FULL TR mean 0.409676, std 0.000601
FULL DIFF mean 0.001910, std 0.000800
Run LightGBM with kfold - done in 62s
............
shift hold out: 2
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
###

In [7]:
feature_importance_df

,feature,importance
0,model,1878.6
1,inner_slot_id,598.4
2,make,207.6
3,advert_name,152.6
4,osv,100.6
5,creative_id,76.6
6,creative_type,57.0
7,f_channel,50.4
8,adid,46.2
9,advert_industry_inner,42.8
